<a href="https://colab.research.google.com/github/shing100/notebooks/blob/main/DPO_Zephyr_Unsloth_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "Runtime" and press "Run all" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join our Discord if you need help!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [DPO data prep](#Data), and how to [train via `DPOTrainer`](#Train).
To learn more about DPO, read TRL's [blog post](https://huggingface.co/blog/dpo-trl). We follow [Huggingface's Alignment Handbook](https://github.com/huggingface/alignment-handbook) to replicate [Zephyr](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta).

In [10]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* DPO requires a model already trained by SFT on a similar dataset that is used for DPO. We use `HuggingFaceH4/mistral-7b-sft-beta` as the SFT model. Use this [notebook](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing) first to train a SFT model.
* [**NEW**] We make Gemma 6 trillion tokens **2.5x faster**! See our [Gemma notebook](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

In [11]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [12]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<a name="Data"></a>
### Data Prep
We follow Huggingface's [Alignment Handbook](https://github.com/huggingface/alignment-handbook) for [Zephyr](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta) and use the [Ultra Feedback dataset](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized), and sample 0.5% of it to speed things up. You can sample the full dataset for a full run.

In [28]:
dataset = load_dataset("CarrotAI/Korea-WebText-Dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'chosen', 'rejected'],
        num_rows: 1009
    })
})

We shall print a random item from the dataset

In [29]:
import pprint
row = raw_datasets["train"][8]
pprint.pprint(row["instruction"])
pprint.pprint(row["chosen"])
pprint.pprint(row["rejected"])

'김도형 씨의 트럭커 경력에서 가장 큰 도전이나 순간은 무엇이었을까요?'
(' 김도형 씨의 트럭커 경력에서 가장 큰 도전이나 순간은 무엇이었는지 알려주세요. 그러면 제가 답변을 드릴 수 있습니다. \n'
 '\n'
 '예를 들어, 김도형 씨가 어려운 운전 환경이나 위험한 상황에 처했던 경험, 혹은 극심한 피로와 싸워야 했던 경험 등을 알려주시면 좀 더 '
 '자세한 답변을 드릴 수 있습니다. \n'
 '\n'
 '더 자세한 정보를 알려주시면 김도형 씨의 트럭커 경력에 대한 답변을 드릴 수 있습니다. \n')
('김도형 씨의 트럭커 경력에서 가장 큰 도전 중 하나는 초기 단계에서의 경제적 부담과 가족과의 거리를 극복하는 것이었습니다. 그는 자신의 '
 '가족을 먹여살리기 위해 트럭커로서의 직업을 선택했지만, 이 길을 택하기 위해 교육비를 지불하고 자격증을 취득해야 했습니다. 또한, '
 '초기에는 개인 차량이 없어 트럭킹 회사에서 차량을 빌리고 매달 리스비를 갚아야 했으며, 이로 인해 장거리 운전을 소화하며 몇 주간 집에 '
 '들어가지 못했습니다. 이는 그의 손이 떨리고 정말 힘들었다고 회상하는 순간이었습니다. 특히 그의 와이프가 첫 아들을 낳은 시기에 혼자서 '
 '육아를 하느라 힘들었을 것이라는 걱정과 미안함을 느끼는 순간은 그의 책임감과 가족을 향한 사랑을 보여주는 중요한 순간이었습니다. 이러한 '
 '경제적 부담과 가족과의 거리를 극복하며 트럭커로서의 경력을 쌓아가는 과정은 김도형 씨에게 있어 큰 도전이자 성장의 계기가 되었습니다.')


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [36]:
def generate_prompt(example):
    prompt = example['instruction']
    rejected = example['rejected']
    chosen = example['chosen']

    example['prompt'] = f"<bos><start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    example['rejected'] = f"{rejected}<end_of_turn>\n<eos>"
    example['chosen'] = f"{chosen}<end_of_turn>\n<eos>"

    return example

In [37]:
transformed_dataset = dataset.map(generate_prompt)

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

In [38]:
transformed_dataset['train'][0]

{'instruction': '고혈압이 무엇이며, 왜 치료를 게을리하면 안 되는지 설명해주세요.',
 'chosen': '## 고혈압이란 무엇일까요?\n\n고혈압은 혈액이 혈관 벽에 가하는 압력이 지속적으로 높은 상태를 말합니다. 마치 물이 파이프를 힘차게 흐르는 것처럼, 혈액이 혈관을 지나치게 강하게 흐르면 혈관 벽에 부담을 주어 여러 질병을 유발할 수 있습니다.\n\n**고혈압의 위험성**\n\n고혈압은 **조용한 살인자**라고 불릴 만큼 위험한 질병입니다. 초기에는 증상이 거의 없어 자신이 고혈압인지 모르는 경우가 많지만, 방치하면 심각한 합병증을 유발할 수 있습니다.\n\n**고혈압으로 인한 합병증**\n\n* **심혈관 질환:** 심장마비, 뇌졸중, 심부전 등 심각한 심혈관 질환의 위험을 높입니다.\n* **신장 질환:** 신장 기능 저하를 초래하여 만성 신부전으로 이어질 수 있습니다.\n* **눈 질환:** 망막 손상, 시력 저하, 실명으로 이어질 수 있습니다.\n* **뇌 질환:** 뇌출혈, 뇌경색 등 뇌졸중의 위험을 높입니다.\n* **혈관 질환:** 동맥경화증, 말초혈관 질환 등을 악화시킬 수 있습니다.\n\n**고혈압 치료의 중요성**\n\n고혈압은 완치가 불가능하지만, **꾸준한 관리를 통해 합병증을 예방하고 건강을 유지할 수 있습니다.**\n\n**고혈압 치료는 다음과 같은 이유로 중요합니다.**\n\n* **합병증 예방:** 고혈압 치료를 통해 혈압을 조절하면 심혈관 질환, 신장 질환, 뇌 질환 등 심각한 합병증을 예방할 수 있습니다.\n* **삶의 질 향상:** 혈압 조절은 혈관 건강을 개선하고 삶의 질을 높이는 데 도움이 됩니다.\n* **수명 연장:** 고혈압 치료는 심혈관 질환으로 인한 사망률을 낮추고 수명을 연장하는 데 효과적입니다.\n\n**고혈압 관리를 위한 노력**\n\n* **정기적인 혈압 측정:** 가정에서 혈압을 측정하고 의사의 지시에 따라 정기적으로 병원을 방문하여 혈압을 체크해야 합니다.\n* **건강한 식습관 유

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Train"></a>
### Train the DPO model
Now let's use Huggingface TRL's `DPOTrainer`! More docs here: [TRL DPO docs](https://huggingface.co/docs/trl/dpo_trainer). We do 3 epochs on 0.5% of the dataset to speed things up.

In [ ]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = raw_datasets["train"],
    # eval_dataset = raw_datasets["test"],
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:294: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/309 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.693100,0.000000,0.000000,0.000000,0.000000,-297.338806,-218.968842,-2.758142,-2.924523
2,0.693100,0.000000,0.000000,0.000000,0.000000,-237.602417,-217.613892,-2.731790,-2.913610
3,0.692200,0.001937,-0.000008,0.625000,0.001945,-172.792877,-202.709259,-2.464616,-2.728198
4,0.692700,0.000855,-0.000130,0.750000,0.000985,-117.745728,-170.711212,-2.591983,-2.805891
5,0.692500,0.001272,-0.000044,0.500000,0.001315,-197.045288,-338.722626,-2.541809,-2.483452
6,0.693700,-0.000155,0.001034,0.500000,-0.001189,-279.681396,-188.683563,-2.870259,-2.633069
7,0.692600,-0.001525,-0.002655,0.625000,0.001130,-157.678253,-236.215118,-2.510528,-2.838598
8,0.692500,0.003246,0.001937,0.500000,0.001309,-451.776550,-493.112579,-2.677159,-2.677977
9,0.693600,-0.003702,-0.002773,0.500000,-0.000929,-198.079987,-405.686523,-2.366659,-2.640016
10,0.685900,0.009754,-0.004801,0.750000,0.014555,-192.067322,-238.161560,-2.735095,-2.766379


TrainOutput(global_step=114, training_loss=0.397163258179238, metrics={'train_runtime': 4017.5003, 'train_samples_per_second': 0.231, 'train_steps_per_second': 0.028, 'total_flos': 0.0, 'train_loss': 0.397163258179238, 'epoch': 2.94})

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Mistral 7b 2x faster [free Colab](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. Gemma 6 trillion tokens is 2.5x faster! [free Colab](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>